In [1]:
!pip uninstall fitz PyMuPDF --yes #why are we installing fitz? SEND HELPS
!pip install PyMuPDF

Found existing installation: PyMuPDF 1.20.1
Uninstalling PyMuPDF-1.20.1:
  Successfully uninstalled PyMuPDF-1.20.1
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached PyMuPDF-1.20.1-cp39-cp39-macosx_10_9_x86_64.whl (7.7 MB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
!open -a Preview "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"  "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf" "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf" 

In [2]:
#NEED TO CHANGE STUFF HERE! 
config = [
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf",
    "output_dir" : "../output/Aaron's_code/",
    "name" : "book-1",
    "page_count" : 642,
    "index" : {
      "range" : range(617, 639) 
    }
    "sections" : {
      "preface" : [1, 78],
      "toponymique" : [49, 76],
      "abbrev" : [77, 78],
      "content" : [79, 607],
      "sample" : [79, 79+10],
      "index" : [617, 639]},
    },
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf",
    "output_dir" : "../output/Aaron's_code/",
    "name" : "book-2",
    "page_count" : 725,
    "sections" : {
      "preface" : [1, 79],
      "abbrev" : [6, 7],
      "content" : [8, 700],
      "sample" : [8, 8+10],
      "index" : [704, 725]},
    },
   {"filename" : "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf",
    "output_dir" : "../output/Aaron's_code/",
    "name" : "book-3",
    "page_count" : 588,
    "sections" : {
      "preface" : [1, 7],
      "abbrev" : [6, 7],
      "content" : [8, 554],
      "sample" : [8, 8+10],
      "index" : [556, 583],
      "familyidx" : [584, 585]},
    },
]


In [3]:
import sys, fitz, re
from pprint import pprint

## Purpose: this cell is a pre-processing step that will create a symlink (e.g. book-1.pdf)
##          that points to the original file, and then will create new PDF documents
##          that are excerpts of subsets of the whole book
##          as named sections such as book-1-index.pdf or book-1-abbrev.pdf
##          The file book-X-sample.pdf contains the first 10 pages of the book-1-content.pdf
##          and the sample is used to speed up development/testing later on

## https://pymupdf.readthedocs.io/en/latest/document.html#Document.delete_pages
##    In general, the performance of this method is dependent on the number of remaining pages
##    NOT on the number of deleted pages
## So, keeping fewer pages is faster, and deleting more pages at once is also faster 

def excerpt(infile, outfile, start, end):
  doc = fitz.open(infile)  # open document
  last_page = doc.page_count
  print(start, end, last_page, outfile)
  deleteme = []
  if start - 1 > 0:
    deleteme.extend(list(range(0, start - 1)))       
  if end < last_page:
    deleteme.extend(list(range(end, last_page)))
  # print(deleteme)
  doc.delete_pages(deleteme)
  doc.save(outfile, garbage=4, clean=True)

import os, os.path
for book in config:
   symlink = book["name"] + ".pdf"
   fname = book["filename"]
   if os.path.exists(book["filename"]) and not os.path.exists(symlink):
      print("Creating symlink: ", symlink, " ->", fname)
   for section in book["sections"]:
      #print(s)
      start, end = book["sections"][section];
      out_fname = book["name"] + "-" + section + ".pdf"
      excerpt(book["filename"], out_fname, start, end)

#!open -a Preview book-[1,2,3]-*.pdf

## Runtime of this cell is ~11m38s [Aaron]
## Runtime of this cell is ~187.9s (about 3 minutes!)[Shide]

1 78 642 book-1-preface.pdf
49 76 642 book-1-toponymique.pdf
77 78 642 book-1-abbrev.pdf
79 607 642 book-1-content.pdf
79 89 642 book-1-sample.pdf
617 639 642 book-1-index.pdf
Creating symlink:  book-2.pdf  -> ../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf
1 79 725 book-2-preface.pdf
6 7 725 book-2-abbrev.pdf
8 700 725 book-2-content.pdf
8 18 725 book-2-sample.pdf
704 725 725 book-2-index.pdf
Creating symlink:  book-3.pdf  -> ../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf
1 7 588 book-3-preface.pdf
6 7 588 book-3-abbrev.pdf
8 554 588 book-3-content.pdf
8 18 588 book-3-sample.pdf
556 583 588 book-3-index.pdf
584 585 588 book-3-familyidx.pdf


In [7]:
# !open -a Preview book-[1,2,3]-*.pdf


In [8]:
!open -a Preview book-[1,2,3]-index.pdf
#SD: First one is corrupted? doesn't open

In [4]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times", 10.929, 0, 4))


Font: 'Times' (serifed, proportional), size 10.929, color #000000


In [5]:
import sys, fitz, re
from pprint import pprint
# fname = "book-1-sample.pdf"
fname = "book-1.pdf"
doc = fitz.open(fname)  # open document

from fitz.utils import getColor
#blue = getColor("aliceblue")

# txtpgoffset = 77
txtpgoffset = 0
# start_page = 1 + txtpgoffset
start_page = 1
#end_page = start_page + 11
#end_page = start_page + 150
#end_page = start_page + 530
end_page = doc.page_count
items = []
mode = "searching"
print(mode)
for i in range(start_page, end_page):
    page = doc[i]
    # read page text as a dictionary, suppressing extra spaces in CJK fonts
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                if z==0 and mode == "found":
                    mode = "spanning"
                #s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                t = s["text"]
                ## FIXME: [83, 'N  Cystopteris  filix-fragilis  (L.) Borb. —  Polypodiumfilix-fragile  L.,  Cyst,  fragilis']
                ## FIXMECLASS: OCR Error; a spurious mark in the text preceding the genus is interpreted as a letter 'N'
                if z==0 and mode != "found" and re.search(r"Times-Bold\b", str(face)) and s["size"] > 10.0:
                        r_rect = fitz.Rect(lr)
                        r_high = fitz.Rect(s["bbox"])
                        joined = " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))  ## FIXMEDESC: ideally this simplistic code would work if not for OCR Errors
                        item = joined
                        # found = re.match(r"(?P<genus>([A-Z]\w+) (?P<species>\w+)", joined)
                        # d = found.groupdict()
                        matches = re.match(r"([A-Z]\w+) (\w+)", joined)
                        warning = "Possible OCR Error"
                        if matches and len(matches.groups()) == 2:
                            found = matches.groups()
                            warning = "Normal"
                            mode = "found"
                            # item = f"""${d['genus']} ${d['species']}"""
                            item = " ".join(found)
                        else:
                            warning = "Possible OCR Error"

                        print([i, x, y, z, item, warning])
                        annot_high = page.add_highlight_annot(r_high)
                        items.append([t, r_rect, r_high, i, z,s, y,l, x,b, warning])
print(len(items))

#NOTE: this block is finding the binomials.
#       they need to be first letters of a span (why not line?)
#       they need to match the binomial paterns : first word start with an upper case letter and rest lower with one space"([A-Z]\w+) (\w+)"
#       Font is greater than 10 (not sure why)
#       Font is r"Times-Bold\b"
#       but there's something wrong because the following shouldn't be picked up:
#               [34, 0, 0, 0, 'r', 'Possible OCR Error']
#               [59, 9, 0, 0, 'J', 'Possible OCR Error']
#               [77, 8, 5, 0, 'JL', 'Possible OCR Error']
#print(i, items[0][:3])


searching
[34, 0, 0, 0, 'r', 'Possible OCR Error']
[59, 9, 0, 0, 'J', 'Possible OCR Error']
[77, 8, 5, 0, 'JL', 'Possible OCR Error']
[78, 8, 0, 0, 'Lycopodium cernuum', 'Normal']
[79, 1, 0, 0, 'Selaginella denticulata', 'Normal']
[79, 16, 0, 0, 'Isoetes hystrix', 'Normal']
[79, 24, 1, 0, 'Isoetes olympica', 'Normal']
[80, 13, 0, 0, 'Equisetum maximum', 'Normal']
[80, 18, 0, 0, 'Equisetum palustre', 'Normal']
[80, 25, 0, 0, 'Equisetum ramosissimum', 'Normal']
[81, 9, 0, 0, 'Ophioglossum vulgatum', 'Normal']
[81, 11, 3, 0, 'Ophioglossum lusitanicum', 'Normal']
[81, 21, 0, 0, 'Osmunda regalis', 'Normal']
[82, 9, 0, 0, 'Gymnogramma leptophylla', 'Normal']
[82, 17, 0, 0, 'Cheilanthes pteridioides', 'Normal']
[83, 28, 0, 0, 'Âdiantum capillus-veneris I (PL III, n. 4). — 2|. Rhizome rampant à écailles', 'Possible OCR Error']
[84, 10, 0, 0, 'Pteris longifolia', 'Normal']
[84, 16, 0, 0, 'Pteridium aquilmum', 'Normal']
[85, 1, 0, 0, 'Athyrium filix', 'Normal']
[85, 12, 0, 0, 'Dryopterîs aculeat

In [6]:
## This cell requires ~20 mins to calculate results from the first book

import math
doc = fitz.open(fname)  # open document
#subset = items[:6]
subset = items
boxes = []
for it, item in enumerate(subset):
    mode = "searching"
    if (it >= len(subset) - 1):
        break
    next_item = subset[it + 1]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    [t, rr, rh, i, z,s, y,l, x,b, warning] = [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0]
    [t1, rr1, rh1, i1, z1,s1, y1,l1, x1,b1, warning1] = next_item
    page = doc[i0]
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    blocks = blocks[x:]
    r_rect = rr0
    
    #print(t0)
    # if t0 == "Isoetes olympica":
    #print([int(it), len(subset), rr0, t0, "\n", t1])
    
    annot_rect = page.add_rect_annot(rr0)
    annot_rect.set_border(width=1, dashes=[1,2])
    annot_rect.update()

    
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            s = l["spans"][0]
            #for z,s in enumerate(l["spans"]):  # iterate through the text spans
            this_line = " ".join(list(map(lambda x: x["text"], l["spans"])))
            if mode != "found":
                if re.search("Aire g", this_line):
                    mode = s["font"] + " " + str(s["size"])
                    r_rect = r_rect.include_rect(lr)
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                elif lr.intersects(rr1):                    
                    mode = "found"
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                else:
                    if mode != "searching":
                        (font, size) = mode.split(" ")
                        isize = round(math.ceil(float(size)))
                        if s["size"] >= float(isize):
                            mode = "found"
                    if mode != "found":
                        r_rect = r_rect.include_rect(lr)
                        # if y >= y0:
                        #     r_rect = r_rect.include_rect(lr)
                    
    r_high = rh0
    annot_rect = page.add_rect_annot(r_rect)
    annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, start_page + it, r_rect, item]) . ## FIXME: thinking it should be i0 not start_page + it
    boxes.append([t0, i0, r_rect, item])


# print(boxes[0])
print(len(boxes))

#doc.delete_pages(end_page, doc.page_count - 1)
#doc.delete_pages(0, start_page - 1)
marked_epithet_fname = "marked-pages-" + doc.name
doc.save(marked_epithet_fname, garbage=4, clean=True)
#!open -a Preview marked-pages-book-1-sample.pdf
!open -a Preview marked-pages-book-1.pdf



1064


In [7]:
import pprint
pp = pprint.PrettyPrinter(compact=True)

import math
doc = fitz.open(fname)  # open document
results = {}
for it, thing in enumerate(boxes):
    mode = "searching"
    #print(thing)
    [name, pageno, rrr, item] = thing
    page = doc[pageno]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", clip=rr0, flags=flags)["blocks"]    


    print(pageno - txtpgoffset, i0, t0)
    results[t0] = {}
    results[t0]["name"] = t0
    results[t0]["name_rects"] = [rr0]
    results[t0]["pdf_page"] = pageno
    results[t0]["book_page"] = pageno - txtpgoffset
    key = "Extra"
    mode = "searching"
    margins = [[(math.ceil(l["bbox"][0]), l["spans"][0]["text"].split(" ")[0]) for l in filter(lambda bl: bl.get("spans"), b["lines"])] for b in blocks]
    margins = [m for n in margins for m in n]
    targets = list(map(lambda n: n[0], filter(lambda m: m[1] in ["L.", "S.", "Aire"] , margins))) or [blocks[0]["lines"][0]["bbox"][0]]
    margins_max = max(targets or margins)
    margins_min = min(list(map(lambda n: n[0], margins)))

    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            xloc = min(margins_max, math.ceil(lr.x0))
            this_line = re.sub(r"\s+", " ", " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))).strip()
            # matches = re.match("^(?P<label>L\.|S\.|Aire g\w+\.|Not\w+\.|Fleurs|Fructif\.|Fructific|Floraison\w+)\s", this_line)
            #### FIXME: Phyllitis scolopendrium (L.) Newm. —Asplenium scolopendrium L., Scolopendrium
            #### FIXME: Juniper
            matches = re.match("^(?P<label>" + t0 + "|.*?)(\s|$)", this_line)
            if matches:
                d = matches.groupdict()
                if margins_max - xloc < 5:
                    key = d["label"]
                    if ' ' in key:
                        key = "Description"
                    elif key not in ["Description", "L.", "S.", "Aire"]:
                        key = "Extra"
                    elif key in ["Description", "L.", "S.", "Aire"]:
                        pass
                    else:
                        print("UNHANDLED: " + key)
                        key = "Extra"
                frag = results[t0].get(key) or ""
                results[t0][key] = " ".join([frag.strip(), this_line.strip()]).strip()
                key_rects = key + "_rects"
                if not results[t0].get(key_rects):
                    results[t0][key_rects] = []    
                results[t0][key_rects].append(lr)
                #print([x, y, xloc, mode, this_line])
            else:
                print(["ERROR:", x, y, xloc, mode, this_line])
    # pp.pprint(results[t0])
    print()
    #break
                    
    # r_high = rh0
    # annot_rect = page.add_rect_annot(r_rect)
    # annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, r_rect, item])




34 34 r

59 59 J

77 77 JL

78 78 Lycopodium cernuum

79 79 Selaginella denticulata

79 79 Isoetes hystrix

79 79 Isoetes olympica

80 80 Equisetum maximum

80 80 Equisetum palustre

80 80 Equisetum ramosissimum

81 81 Ophioglossum vulgatum

81 81 Ophioglossum lusitanicum

81 81 Osmunda regalis

82 82 Gymnogramma leptophylla

82 82 Cheilanthes

83 83 Âdiantum capillus-veneris

84 84 Pteris longifolia

84 84 Pteridium aquilmum

85 85 Athyrium

85 85 Dryopterîs aculeata

85 85 Dryopteris

86 86 Blechnum

87 87 Ceterack officinarum

88 88 Asplenium bourgaei

88 88 Asplenium

88 88 Asplenium ruta-muraria

88 88 Asplenium

88 88 Asplenium

89 89 Phyllitis scolopendrium

89 89 Phyllitis hemionitis

90 90 Polypodium vulgare L.

90 90 Salvinia natans

92 92 Abies cilicica

92 92 Cedrus libani

93 93 Pinus pinea

93 93 Pina

93 93 Pinus halepensis

94 94 Cupressus sempervirens

95 95 Juniperus oxycedrus

96 96 Juniper

96 96 Juniperus

96 96 Arceuthos drupacea

97 97 Ephedra campylopoda

97 97 

In [8]:
from tqdm import tqdm
import math
# marked_fname = "marked-pages-book-1-sample.pdf"
marked_fname = "marked-pages-book-1.pdf"
#doc = fitz.open(fname)  # open document
doc = fitz.open(marked_fname)  # open document
print(len(results))

from fitz.utils import getColorList
cl = getColorList()
from fitz.utils import getColor

pink = getColor("lightpink")
green = getColor("aquamarine")
blue = getColor("lightskyblue")
gray = getColor("whitesmoke")
yellowish = getColor("antiquewhite")
fills= [pink, green, blue, gray, yellowish]

fc = {"L.":pink, "S.":blue, "Aire":green, "Description":gray, "Extra":yellowish}
for it, name in enumerate(tqdm(results.keys())):
    if it == 500:
        break
    item = results[name]
    #print(item)
    #print(item["Description"])
    book_page = item["book_page"]
    pdf_page = item["pdf_page"]
    #print([it, name, book_page, pdf_page])
    page = doc[book_page]#doc[book_page - 1]
    for k in filter(lambda x: not re.search(r"_page|_rect", x), item.keys()):
        rects = item[k + "_rects"]
        for r in rects:
            #print(k, r)
            
            if not page.rect.intersects(r):
                #print(["ERROR: ", k, r])
                pass
            else:
                if k in ["L.", "S.", "Aire", "Description", "Extra"]:
                    #print(["HIGHL: ", fc[k], k, r])
                    annot = page.add_highlight_annot(fitz.Rect(r))
                    annot.set_colors(stroke=fc[k])
                    annot.update()
                    #annot_rect = page.add_rect_annot(fitz.Rect(r))
                    #annot_rect.set_colors(stroke=green)
                    #annot_rect.set_border() #(width=1, dashes=[1,2])
                    #annot_rect.update()
                    #annot_rect.update(fill_color=red)


# doc.delete_pages(end_page, doc.page_count - 1)
# doc.delete_pages(0, start_page - 1)
doc_fname = "results-" + doc.name
doc.save(doc_fname, garbage=4, clean=True)
print(doc_fname)
# !open -a Preview results-marked-pages-book-1-sample.pdf
!open -a Preview results-marked-pages-book-1.pdf



862


 58%|█████▊    | 500/862 [00:01<00:01, 262.55it/s]


results-marked-pages-book-1.pdf


In [1]:
import pandas as pd

df = pd.DataFrame.from_dict(results.values()).fillna('')
df = df[df.columns.drop(list(df.filter(regex='_rects')))]
df.to_csv("book-1.csv", index=False)
df

NameError: name 'results' is not defined